In [1]:
from platform import python_version

print(python_version())

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import balanced_accuracy_score

In [3]:
import gensim
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [4]:
import re

In [5]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")  # Punkt Sentence Tokenizer
nltk.download("averaged_perceptron_tagger")  # Part of Speech Tagger
nltk.download("wordnet")  # a lexical database of English; useful for synonyms, hyponyms, etc.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MondayPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MondayPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MondayPC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MondayPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [7]:
import random

In [8]:
stop = set(stopwords.words('english'))
random.sample(stopwords.words('english'), 10)

['am',
 'down',
 'be',
 'where',
 "doesn't",
 'or',
 'only',
 "you'll",
 'herself',
 'in']

# Loading Data

In [9]:
liar_dataset_train = pd.read_csv('../data/LIAR-PLUS/train2.tsv', sep='\t', header = None)
liar_dataset_test = pd.read_csv('../data/LIAR-PLUS/test2.tsv', sep='\t', header = None)
liar_dataset_valid = pd.read_csv('../data/LIAR-PLUS/val2.tsv', sep='\t', header = None)
liar_dataset = pd.concat([liar_dataset_train, liar_dataset_test, liar_dataset_valid], axis = 0).reset_index(drop = True)

In [10]:
liar_dataset = liar_dataset.iloc[:, [2, 3, 15]]
liar_dataset = liar_dataset.rename(columns = {2: 'label', 3: 'statements', 15: 'justification'})

In [11]:
liar_dataset.dropna(inplace=True)
liar_dataset.reset_index(inplace=True, drop=True)

In [12]:
liar_dataset

,label,statements,justification
0,false,Says the Annies List political group supports ...,That's a premise that he fails to back up. Ann...
1,half-true,When did the decline of coal start? It started...,"Surovell said the decline of coal ""started whe..."
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",Obama said he would have voted against the ame...
3,false,Health care reform legislation is likely to ma...,The release may have a point that Mikulskis co...
4,half-true,The economic turnaround started at the end of ...,"Crist said that the economic ""turnaround start..."
...,...,...,...
12687,half-true,"For the first time in more than a decade, impo...","In 2009, 17 percent of the U. S. 's oil import..."
12688,mostly-true,Says Donald Trump has bankrupted his companies...,"Clinton said, Trump has ""bankrupted his compan..."
12689,true,"John McCain and George Bush have ""absolutely n...","""I don't think that there should be a mandate ..."
12690,false,A new poll shows 62 percent support the presid...,But the poll doesn't say that. Several days af...


In [13]:
liar_dataset['label'] = liar_dataset['label'].replace({
    'false' : 0,
    'barely-true' : 0,
    'pants-fire' : 0,
    'half-true' : 1,
    'mostly-true' : 1,
    'true' : 1
})

In [14]:
liar_dataset

,label,statements,justification
0,0,Says the Annies List political group supports ...,That's a premise that he fails to back up. Ann...
1,1,When did the decline of coal start? It started...,"Surovell said the decline of coal ""started whe..."
2,1,"Hillary Clinton agrees with John McCain ""by vo...",Obama said he would have voted against the ame...
3,0,Health care reform legislation is likely to ma...,The release may have a point that Mikulskis co...
4,1,The economic turnaround started at the end of ...,"Crist said that the economic ""turnaround start..."
...,...,...,...
12687,1,"For the first time in more than a decade, impo...","In 2009, 17 percent of the U. S. 's oil import..."
12688,1,Says Donald Trump has bankrupted his companies...,"Clinton said, Trump has ""bankrupted his compan..."
12689,1,"John McCain and George Bush have ""absolutely n...","""I don't think that there should be a mandate ..."
12690,0,A new poll shows 62 percent support the presid...,But the poll doesn't say that. Several days af...


In [15]:
train_dataset = liar_dataset

In [16]:
train_dataset.head()

,label,statements,justification
0,0,Says the Annies List political group supports ...,That's a premise that he fails to back up. Ann...
1,1,When did the decline of coal start? It started...,"Surovell said the decline of coal ""started whe..."
2,1,"Hillary Clinton agrees with John McCain ""by vo...",Obama said he would have voted against the ame...
3,0,Health care reform legislation is likely to ma...,The release may have a point that Mikulskis co...
4,1,The economic turnaround started at the end of ...,"Crist said that the economic ""turnaround start..."


# Data Preprocessing and Data Preparation

In [17]:
train_dataset = train_dataset.fillna("null data")

### Removing stopwords

In [18]:
stop_words = stopwords.words('english')

In [19]:
if_stopwords = True

In [20]:
if if_stopwords:
    train_dataset["statements"] = train_dataset["statements"].str.lower().str.replace("’", "'")
    train_dataset["statements"] = train_dataset["statements"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    train_dataset["justification"] = train_dataset["justification"].str.lower().str.replace("’", "'")
    train_dataset["justification"] = train_dataset["justification"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [21]:
train_dataset.head()

,label,statements,justification
0,0,says annies list political group supports thir...,that's premise fails back up. annie's list mak...
1,1,decline coal start? started natural gas took s...,"surovell said decline coal ""started natural ga..."
2,1,"hillary clinton agrees john mccain ""by voting ...",obama said would voted amendment present. thou...
3,0,health care reform legislation likely mandate ...,release may point mikulskis comment could open...
4,1,economic turnaround started end term.,"crist said economic ""turnaround started end te..."


### Preprocessing

In [22]:
preprocessing_text_fn = {
    "no_punctuation": lambda txt: re.sub(r'[^\w\s]','', txt),
    "no_special_symbols": lambda txt: re.sub('[$,#,&]', '', txt),
    "no_digits": lambda txt: re.sub('\d*', '', txt),
    "no_www": lambda txt: re.sub('w{3}', '', txt),
    "no_urls": lambda txt: re.sub('http\S+', '', txt),
    "no_spaces": lambda txt: re.sub('\s+', ' ', txt),
    "no_single_chars": lambda txt: re.sub(r'\s+[a-zA-Z]\s+', '', txt)
}

In [23]:
def preprocess_text(text, pipeline = preprocessing_text_fn):
    text = str(text)
    for fn in pipeline.keys():
        text = pipeline[fn](text)
        
    return text

In [24]:
STOP_WORDS = [preprocessing_text_fn["no_punctuation"](word) for word in stop_words]
random.sample(stop_words, 20)

['off',
 'on',
 'me',
 'few',
 'into',
 "mightn't",
 'if',
 'when',
 'her',
 'no',
 'these',
 "wasn't",
 "you've",
 'just',
 'll',
 'such',
 'and',
 "you're",
 'their',
 'where']

In [25]:
def tokenize_without_stopwords(text, stop_words=STOP_WORDS):
    word_tokens = word_tokenize(text)
    filtered_sequence = [word for word in word_tokens if not word.lower() in stop_words]
    return filtered_sequence

In [26]:
train_dataset["statements"] = train_dataset["statements"].apply(preprocess_text)
train_dataset["justification"] = train_dataset["justification"].apply(preprocess_text)
train_dataset.head(10)

,label,statements,justification
0,0,says annies list political group supports thir...,thats premise fails back up annies list makes ...
1,1,decline coal start started natural gas took st...,surovell said decline coal started natural gas...
2,1,hillary clinton agrees john mccain by voting g...,obama said would voted amendment present thoug...
3,0,health care reform legislation likely mandate ...,release may point mikulskis comment could open...
4,1,economic turnaround started end term,crist said economic turnaround started end ter...
5,1,chicago bears starting quarterbacks last years...,vos specifically used word fired means faculty...
6,0,jim dunnam lived district represents years now,determining would take significant detective w...
7,1,im person stage worked actively last year pass...,however bill another one sponsored majority le...
8,1,however took million oregon lottery funds port...,johnson correct many factors played role lotte...
9,1,says gop primary opponents glenn grothman joe ...,considering million figure covers years reason...


### Lemmatization and Stemming

In [28]:
if_lemmatize = True

In [29]:
if if_lemmatize:
    
    import nltk
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    from nltk.stem import WordNetLemmatizer
    
    wnl = WordNetLemmatizer()
    
    train_dataset["statements"] = train_dataset["statements"].str.lower().str.replace("’", "'")
    train_dataset["statements"] = train_dataset["statements"].apply(lambda x: ' '.join([wnl.lemmatize(word) for word in word_tokenize(x)]))
    train_dataset["justification"] = train_dataset["justification"].str.lower().str.replace("’", "'")
    train_dataset["justification"] = train_dataset["justification"].apply(lambda x: ' '.join([wnl.lemmatize(word) for word in word_tokenize(x)]))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MondayPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MondayPC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [30]:
SJ = True

if SJ:
    train_text = (train_dataset['justification'] + " " + train_dataset['statements']).values
else:
    train_text = train_dataset['justification']

In [31]:
train_labels = train_dataset['label'].values

## Embedding with word2vec

In [33]:
# Limit length of each article
max_length = 3300
lengths = np.array([len(x) for x in train_text])
train_text = train_text[lengths < max_length]
train_labels = train_labels[lengths < max_length]

In [34]:
# Check actual max length of an article
article_length = max(np.array([len(x) for x in train_text]))
article_length

3278

In [35]:
# Word2vec model with vector size = 100
vec_size = 100

# workers - number of CPU threads
word_model = gensim.models.Word2Vec(train_text, vector_size = vec_size, window = 5, workers = 12)
word_model.train(train_text, epochs = 10, total_examples = len(train_text))
wv = word_model.wv

In [36]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
vocabulary_size = len(tokenizer.word_index) + 1
encoded_articles = tokenizer.texts_to_sequences(train_text)

In [37]:
padded_articles = pad_sequences(encoded_articles, maxlen = article_length, padding='post')

In [38]:
padded_articles.shape

(12683, 3278)

In [39]:
emb_matrix = np.zeros(shape=(vocabulary_size, vec_size))
for w, i in tokenizer.word_index.items():
    ind = wv.has_index_for(w)
    if ind:
        emb_matrix[i] = wv.get_vector(w)

In [40]:
x_train, x_test, y_train, y_test = train_test_split(padded_articles, train_labels, test_size=0.25)

## LSTM model

In [41]:
from keras.initializers import Constant
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, LSTM
from keras.layers import ReLU
from keras.layers import Dropout
from tensorflow.keras.losses import CategoricalCrossentropy

model=Sequential()
model.add(Embedding(input_dim = vocabulary_size, 
                    output_dim = vec_size,
                    input_length = article_length,
                    embeddings_initializer = Constant(emb_matrix))
         )
model.add(LSTM(32, return_sequences=True))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3278, 100)         2747700   
                                                                 
 lstm (LSTM)                 (None, 3278, 32)          17024     
                                                                 
 flatten (Flatten)           (None, 104896)            0         
                                                                 
 dense (Dense)               (None, 128)               13426816  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                2064      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0

In [43]:
import keras.backend as K
def keras_f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [44]:
model.compile(optimizer=Adam(),loss='binary_crossentropy', metrics=['accuracy'])

## Training

In [45]:
epochs=5
batch_size=128

In [46]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Epoch 1/5
75/75 [==============================] - 13s 154ms/step - loss: 0.6930 - accuracy: 0.5419 - val_loss: 0.6864 - val_accuracy: 0.5572
Epoch 2/5
75/75 [==============================] - 11s 152ms/step - loss: 0.6830 - accuracy: 0.5542 - val_loss: 0.6743 - val_accuracy: 0.5825
Epoch 3/5
75/75 [==============================] - 12s 154ms/step - loss: 0.5666 - accuracy: 0.7130 - val_loss: 0.7002 - val_accuracy: 0.5812
Epoch 4/5
75/75 [==============================] - 12s 156ms/step - loss: 0.3249 - accuracy: 0.8680 - val_loss: 1.0116 - val_accuracy: 0.5276
Epoch 5/5
75/75 [==============================] - 11s 153ms/step - loss: 0.1021 - accuracy: 0.9674 - val_loss: 1.6033 - val_accuracy: 0.5399


## Metrics

In [47]:
from sklearn.metrics import f1_score
def accuracy(predictions, labels):
    predictions[predictions >= 0.5] = 1
    predictions[predictions < 0.5] = 0
    return accuracy_score(labels, predictions)

def flat_f1_score(predictions, labels):
    predictions[predictions >= 0.5] = 1
    predictions[predictions < 0.5] = 0
    return f1_score(labels, predictions)

In [48]:
train_pred = model.predict(x_train)
train_truth = y_train

298/298 [==============================] - 10s 34ms/step


In [49]:
# Accuracy
accuracy(train_pred, train_truth)

0.9965306980656014

In [50]:
# F1 score
flat_f1_score(train_pred, train_truth)

0.9968993704782486

## Applying on test dataset

In [52]:
test_pred = model.predict(x_test)
test_truth = y_test

100/100 [==============================] - 3s 34ms/step


In [58]:
# Accuracy
accuracy(test_pred, test_truth)

0.5398927783033743

In [57]:
# Balanced accuracy
flat_f1_score(test_pred, test_truth)

0.6023439629326792